# Retriever

Once we have data stored in cloudsql, we need to find a way to query the data. This notebook covers how we can create and use the postgresql retriever to perform similarity search.

Start with adding the parameters/constants data:

In [1]:
project_id = "imrenagi-gemini-experiment"  # @param {type:"string"}
region = "us-central1"

instance_name="pyconapac-demo"
database_password = 'testing'
database_name = 'testing'
database_user = 'testing'

embeddings_table_name = "course_content_embeddings"
gemini_embedding_model = "text-embedding-004"

assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

ip_addresses = !gcloud sql instances describe {instance_name} --format="value(ipAddresses[0].ipAddress)"
database_host = ip_addresses[0]

Now let's instantiate embedding service which will be used to create the embedding from the given query:

In [2]:
import vertexai
from langchain_google_vertexai import VertexAIEmbeddings

vertexai.init(project=project_id, location=region)
embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)

Same as previous module, we will start PostgresEngine to connect to CloudSQL instance:

In [3]:
from langchain_google_cloud_sql_pg import PostgresEngine

pg_engine = PostgresEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user=database_password,
    password=database_password,
)

We create the vector store object by using the engine and embedding service we created earlier:

In [4]:
from langchain_google_cloud_sql_pg import PostgresVectorStore

vector_store = PostgresVectorStore.create_sync(
            pg_engine,
            table_name=embeddings_table_name,
            embedding_service=embeddings_service,
        )
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

Let's try with some query:

In [ ]:
retriever.invoke("how to design forgot password?")

In [ ]:
retriever.invoke("how to design security for authentication?")